In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)

import wandb
from huggingface_hub import login, HfApi, create_repo
from pathlib import Path
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from vllm import LLM, SamplingParams
from vllm.lora.request import LoRARequest




/home/012/r/rx/rxh210037/.conda/envs/llmenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 10-20 11:52:42 __init__.py:183] Automatically detected platform cuda.


2025-10-20 11:52:42,218	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
#Wandb & Huggingface keys
hf_token = "***REMOVED***"
wandb_api_key = "**REMOVED***"

In [3]:
# Intitialize Weights & Biases
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to WANDB!")
else:
    print("No wandb key provided. Skipping wandb login.")

if hf_token:
    # Log in to Hugging Face
    login(token=hf_token)
    print("Successfully logged in to Hugging Face!")
else:
    print("Hugging Face token not found in notebook secrets.")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/012/r/rx/rxh210037/.netrc
wandb: Currently logged in as: reyhaneh-rhp7 (reyhaneh-rhp7-university-of-texas-at-dallas) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Successfully logged in to WANDB!
Successfully logged in to Hugging Face!


In [4]:
wandb.init(project="Jigsaw_PartB_Inference_vLLM", name="Jigsaw_PartB_Inference_vLLM")  # you can omit name if not needed


wandb: Detected [openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


In [ ]:
data_folder = Path ("****/jigsaw-agile-community-rules")

test_path = data_folder / "test.csv"
df = pd.read_csv(test_path)
print(f"Dataframe shape: {df.shape}")
df.head()


Dataframe shape: (10, 8)


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...


In [6]:
dataset = Dataset.from_pandas(df) # Convert pandas DataFrame to Hugging Face Dataset


In [7]:
class_names = ["complies", "violates"]

prompts = [
    f"Classify the TEXT by selecting label from the following list: {class_names}. ### TEXT: {text.strip()} ### LABEL:"
    for text in df["body"]
]



In [8]:
BASE_MODEL  = "Qwen/Qwen1.5-7B"

LORA_ADAPTER_PATH  = "./results_qwen_lora/checkpoint-52"


In [9]:
def free_gpu_memory():
    """
    Frees up GPU memory by clearing cache and garbage collecting.
    """
    import torch
    import gc
    
    # Empty CUDA cache
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
        # Get initial GPU memory info
        initial_mem = torch.cuda.memory_allocated()
        
        # Force garbage collection
        gc.collect()
        
        # Get final GPU memory info
        final_mem = torch.cuda.memory_allocated()
        
        print(f"GPU memory freed: {(initial_mem - final_mem) / 1024**2:.2f} MB")
    else:
        print("No GPU available")

In [10]:
free_gpu_memory()


GPU memory freed: 0.00 MB


In [11]:
import torch
torch.cuda.empty_cache()



In [12]:

llm = LLM(
    model=BASE_MODEL,
    dtype="bfloat16",              # string, not torch dtype
    trust_remote_code=True,
    enable_lora=True,
    max_lora_rank=128,
    gpu_memory_utilization=0.6,
    max_model_len=262,             # or larger if you have VRAM
    guided_decoding_backend="outlines",
)

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-20 11:52:49 config.py:520] This model supports multiple tasks: {'classify', 'generate', 'embed', 'reward', 'score'}. Defaulting to 'generate'.
INFO 10-20 11:52:49 llm_engine.py:232] Initializing an LLM engine (v0.7.0) with config: model='Qwen/Qwen1.5-7B', speculative_config=None, tokenizer='Qwen/Qwen1.5-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=262, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen1.5-7B, num_scheduler_steps=1, multi_step_stream_outputs=Tru

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.68it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.41it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.26it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.31it/s]



INFO 10-20 11:52:53 model_runner.py:1115] Loading model weights took 14.3919 GB
INFO 10-20 11:52:53 punica_selector.py:16] Using PunicaWrapperGPU.
INFO 10-20 11:52:54 worker.py:266] Memory profiling takes 0.99 seconds
INFO 10-20 11:52:54 worker.py:266] the current vLLM instance can use total_gpu_memory (47.41GiB) x gpu_memory_utilization (0.60) = 28.45GiB
INFO 10-20 11:52:54 worker.py:266] model weights take 14.39GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 12.60GiB.
INFO 10-20 11:52:55 executor_base.py:108] # CUDA blocks: 1612, # CPU blocks: 512
INFO 10-20 11:52:55 executor_base.py:113] Maximum concurrency for 262 tokens per request: 98.44x
INFO 10-20 11:52:55 model_runner.py:1430] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error o

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:16<00:00,  2.13it/s]

INFO 10-20 11:53:12 model_runner.py:1558] Graph capturing finished in 16 secs, took 0.36 GiB
INFO 10-20 11:53:12 llm_engine.py:429] init engine (profile, create kv cache, warmup model) took 18.74 seconds


In [13]:
# Apply the trained LoRA adapter
lora_request = LoRARequest(
    lora_name="PartB_Inference",
    lora_int_id=1,
    lora_path=LORA_ADAPTER_PATH
)


/tmp/ipykernel_425055/3445125222.py:2: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  lora_request = LoRARequest(


In [14]:
# Define generation (sampling) parameters
sampling_params = SamplingParams(
    temperature=0.0,
    top_p=1.0,
    max_tokens=8,
    stop=["\n"]
)

In [15]:
import re

def normalize_prediction(text):
    text = text.lower().strip()
    if "vio" in text:
        return "violates"
    elif "com" in text:
        return "complies"
    else:
        return "unknown"



In [16]:
# Run batch inference
from tqdm import tqdm
predictions = []

for i in tqdm(range(0, len(prompts), 16), desc="Generating predictions"):
    batch = prompts[i:i+16]
    outputs = llm.generate(batch, sampling_params, lora_request=lora_request)

    for o in outputs:
        raw = o.outputs[0].text
        pred = normalize_prediction(raw)
        predictions.append(pred)


df["predicted_label"] = predictions
print(df)


Generating predictions: 100%|██████████| 1/1 [00:00<00:00,  1.89it/s]

   row_id                                               body  \
0    2029  NEW RAP GROUP 17. CHECK US OUT https://soundcl...   
1    2030  Make your life comfortable. Get up to 15% Disc...   
2    2031  Kickin' ass and selling underwear!\nJust made ...   
3    2032   watch  hooters  best  therein  http://clickan...   
4    2033   bitches  for free  at this point  show all  h...   
5    2034  Top 10 Pentakills in August https://youtu.be/Z...   
6    2035  WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...   
7    2036  [This mixtape is lit FAM!](https://beholdthear...   
8    2037  must be watch movie https://sites.google.com/s...   
9    2038  Free paypal cards here!! https://www.pointspri...   

                                                rule           subreddit  \
0  No Advertising: Spam, referral links, unsolici...         hiphopheads   
1  No legal advice: Do not offer or request legal...           AskReddit   
2  No Advertising: Spam, referral links, unsolici...            gon

In [17]:
from collections import Counter
counts = Counter(predictions)
print("\n🔍 Prediction Summary:")
for k, v in counts.items():
    print(f"{k:10s} : {v}")



🔍 Prediction Summary:
complies   : 4
violates   : 6


In [30]:
df['predicted_label'] = df['predicted_label'].map({'complies': 0, 'violates': 1})
df

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,predicted_label
0,2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...,0
1,2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...,1
2,2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...,1
3,2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...,1
4,2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...,1
5,2034,Top 10 Pentakills in August https://youtu.be/Z...,"No Advertising: Spam, referral links, unsolici...",leagueoflegends,code free tyrande --->>> [Imgur](http://i.imgu...,see tits sweet at this point http://sh.or...,its my asreddit https://www.youtube.com/watch?...,click here for more videos[Russian mafia messe...,0
6,2035,WIN A SAMSUNG SMARTWATCH!\n\nhttp://www.innomd...,"No Advertising: Spam, referral links, unsolici...",AskReddit,"[ENJOY 50,000 BONUS POINTS after $4,000 spent ...",We have high quality Canadian Polymer banknote...,i was on the same ios and try this jailbreak m...,We're streaming Pokemon Veitnamese Crystal RIG...,1
7,2036,[This mixtape is lit FAM!](https://beholdthear...,"No Advertising: Spam, referral links, unsolici...",BlackPeopleTwitter,watch good one hooters there http://https://x5...,bad pussies waiting for you herein http://gour...,[Full HD Movie Online Free](http://www.flickma...,* Computers :http://livematchstreamz.blogspot....,0
8,2037,must be watch movie https://sites.google.com/s...,"No Advertising: Spam, referral links, unsolici...",movies,see there hooters sweet http://url.t.aylo...,"Sign up with my link and get $1,000 of free pr...",The fact that there are pro and cons to everyt...,Oh that's interesting. You should check out my...,0
9,2038,Free paypal cards here!! https://www.pointspri...,"No Advertising: Spam, referral links, unsolici...",pics,she will come your home open her legs with an...,Old and young your girl-next-door make sex wi...,"i think the conjuring 2 is not bad movie ,The...","wow, she's so beautyful\n>[Sakura Vietnam](htt...",1


In [31]:
#  Save predictions
output_path = "./inference_predictions.csv"
df.to_csv(output_path, index=False)
print(f"Saved predictions to {output_path}")

Saved predictions to ./inference_predictions.csv


In [32]:
# Finish wandb run
wandb.finish()